Projeto TCC
Hugo Antonio de Azevedo Lousa

### Declaração das funções necessárias para a leitura dos dados.

In [ ]:
# carga bibliotecas
import os
import pandas as pd
import numpy as np
from simpledbf import Dbf5
import pyarrow as pa
import pyarrow.parquet as pq
import glob
from joblib import dump
from shutil import move
import scipy.stats as stats
import math
 
from IPython.core.interactiveshell import InteractiveShell
#apresenta todos os prints de uma célula. pois o default é apresentar apenas o último.
InteractiveShell.ast_node_interactivity = "all"
#para retornar ao modo default (mostra apenas o último print), descomentar linha abaixo
#InteractiveShell.ast_node_interactivity = "last_expr"

In [ ]:
#valores das variáveis base
base_data_path = 'Dados'
parquet_folder_name = 'Parquet'
leitos_folder_name = 'Leitos'
estabelecimentos_folder_name = 'Estabelecimentos'
producao_folder_name = 'Producao'
territorial_folder_name = 'Territorial'
CID_folder_name = 'CID'
lidos_folder_name = 'Lidos'
joblibs_folder_name = 'Joblib'
joblib_file_name = 'dados_preparados.joblib'
joblib_n_aih = 'dados_n_aih.joblib'
codigo_paraiba = '25'

In [ ]:
def carrega_arquivos_dataframe(local = base_data_path, tipo = 'dbf', encoding = 'iso8859-1', separador = ';', nome = None):

    df = None
    if nome == None:
        arquivos = glob.glob(os.path.join(local, '*.{}'.format(tipo)))
        frames = []
        print('Iniciando importação.')
        print(f'Quantidade de arquivos dbf encontrados = {len(arquivos)}')
        for arquivo in arquivos:
            print(f'Importando arquivo: {arquivo}.')    
            frames.append(Dbf5(arquivo, codec= encoding).to_dataframe())
            
        print('Concatenando arquivos, aguarde...')
        df = pd.concat(frames)
        print('Importação concluída!')
        
    elif tipo == 'csv':
        
        df = pd.read_csv(os.path.join(local, '{}.{}'.format(nome, tipo)), dtype= str, encoding=encoding, sep=separador)
        
    elif tipo == 'dbf':
        
        df = Dbf5(os.path.join(local, '{}.{}'.format(nome, tipo)), codec= encoding).to_dataframe()

    return df

In [ ]:
#Função que tem como objetivo ler os arquivos .dbf do SIHSUS, contendo a produção hospitalar
def obtem_producao_hospitalar():
    sih_producao_bruta = carrega_arquivos_dataframe(local=os.path.join(base_data_path, producao_folder_name), tipo = 'dbf')
    return sih_producao_bruta

In [ ]:
# leitura dados dos municipios, retornando CO_MUNICIP', 'IN_POBREZA', 'REGIAO_SAUDE', 'CLASSE_POP', 
# 'QTD_MUNICIPIOS_REGIAO_SAUDE', 'POPULACAO_REGIAO_SAUDE
def ler_arquivos_territoriais():
#Dicionário de dados ftp://arpoador.datasus.gov.br/territorio/doc/bases_territoriais.pdf
#fonte do arquivo ftp://arpoador.datasus.gov.br/territorio/tabelas/
    #lê o arquivo tb_municip no df municipios
    municipios = carrega_arquivos_dataframe(local=os.path.join(base_data_path, territorial_folder_name), nome = 'tb_municip')
    #lê o arquivo tb_regsaud no df regiao_saude
    regiao_saude = carrega_arquivos_dataframe(local=os.path.join(base_data_path, territorial_folder_name), nome = 'tb_regsaud', encoding= 'utf-8')
    #lê o arquivo de relacionamento rl_municip_regsaud no df rel_regiao_saude
    rel_regiao_saude = carrega_arquivos_dataframe(local=os.path.join(base_data_path, territorial_folder_name), nome = 'rl_municip_regsaud', encoding= 'utf-8')
    #lê o arquivo ibge_populacao no df mun_populacao
    mun_populacao = carrega_arquivos_dataframe(local=os.path.join(base_data_path, territorial_folder_name), nome= 'ibge_populacao', tipo = 'csv', encoding = 'utf-8')
    
    #cria o df qtd_municipios_regiao, contendo os valores distintos da coluna CO_REGSAUD e 
    #número de ocorrência para cada um desses valores, reiniciando-se o índice.
    #isso representa a quantidade de municípios para cada região de saúde
    qtd_municipios_regiao = pd.DataFrame(rel_regiao_saude['CO_REGSAUD'].value_counts()).reset_index()
    #nomeia as colunas do df 
    qtd_municipios_regiao.columns = ['CO_REGSAUD', 'QTD_MUNICIPIOS']
    
    #Realiza o merge (semelhante ao INNER JOIN do SQL) do df qtd_municipios_regiao ao df regiao_saude
    regiao_saude = regiao_saude.merge(qtd_municipios_regiao, on='CO_REGSAUD')

    #filtra apenas regiões de saúde ativas
    regiao_saude = regiao_saude[regiao_saude['CO_STATUS']=='ATIVO']

    #Trecho a seguir contabiliza a populacao de cada região de saúde
    
    #copia-se o conteúdo do df rel_regiao_saude (colunas "CO_MUNICIP","CO_REGSAUD") para o df populacao_regiao_saude
    #a ser preenchido a seguir com a informação de população
    populacao_regiao_saude = rel_regiao_saude.copy()
    #seleciona-se apenas as colunas Cod Municipio' e 'POP EST'
    municipio_populacao = mun_populacao.loc[:, ('Cod Municipio', 'POP EST')]
    #substituindo a série que contém a coluna Cod Municipio, originalmente com código do município com 7 dígitos,
    #por pela mesma série, só que com o código do município apenas com os 6 primeiros dígitos (o stop do slice é exclusivo)
    municipio_populacao.loc[:,('Cod Municipio')] = mun_populacao.loc[:,'Cod Municipio'].str.slice(start=0, stop=6, step=None).values
    #junção (INNER JOIN do SQL) entre o df populacao_regiao_saude (chave CO_MUNICIP) e o df municipio_populacao (chave Cod Municipio)
    populacao_regiao_saude = populacao_regiao_saude.merge(municipio_populacao, how='inner', left_on='CO_MUNICIP', right_on='Cod Municipio')
    #dropa a coluna Cod Municipio após a junção, com inplace=True para garantir que é aquele df que terá a coluna apagada 
    #e não uma eventual cópia
    populacao_regiao_saude.drop(columns=['Cod Municipio'], inplace=True)
    #coluna POP EST do df populacao_regiao_saude recebe a mesma coluna, agora com o tipo int, pois o valor é numérico
    populacao_regiao_saude['POP EST'] = populacao_regiao_saude['POP EST'].astype(str).astype(int)
    #seleciona-se no df populacao_regiao_saude as colunas 'CO_REGSAUD' e 'POP EST', agrupa-se por CO_REGSAUD e somando-se os
    # valores de POP EST, chegando-se à população total de cada região de saúde.
    populacao_regiao_saude = populacao_regiao_saude.loc[:,('CO_REGSAUD', 'POP EST')].groupby(['CO_REGSAUD'])['POP EST'].sum()
    #recria o dataframe populacao_regiao_saude, resetando o índice CO_REGSAUD
    populacao_regiao_saude = pd.DataFrame(populacao_regiao_saude).reset_index(level=['CO_REGSAUD'])

    #Começando a construir o df df_municipios, que será o retorno dessa função
    #Mantendo apenas os municípios com NU_ORDEM=1 (1: municípios ativos (CO_STATUS = ATIVO))
    df_municipios = municipios[municipios.NU_ORDEM == '1'].reset_index(drop=True)

    #Seleciona cod municipio e classe da populacao
    # Classe população é código que vai de 1 a 7, conforme faixas de população no município, definidas pelo IBGE
    # Ex.: São Paulo => 7 - Maior que 500000, Abadiânia => 3 - 10001 até 20000
    classe_populacao = mun_populacao[['Cod Municipio', 'CLASSE POP']]
    classe_populacao.columns = ['CO_MUNICIP', 'CLASSE_POP']
    classe_populacao = classe_populacao.reset_index(drop=True)

    #junção entre os municipios e o codigo da regiao de saúde de cada um deles, usado a chave CO_MUNICIP, para em seguida
    #trazer mais informações da região de saúde
    df_municipios = pd.merge(df_municipios, rel_regiao_saude, how='left', on='CO_MUNICIP')

    #seleciona-se apenas as colunas 'CO_REGSAUD', 'DS_ABREV', 'QTD_MUNICIPIOS' do df regiao_saude (verificar vírgula sobrando)
    regiao_saude = regiao_saude[['CO_REGSAUD', 'DS_ABREV', 'QTD_MUNICIPIOS', ]]
    #logo após as renomeia
    regiao_saude.columns = ['CO_REGSAUD', 'REGIAO_SAUDE', 'QTD_MUNICIPIOS_REGIAO_SAUDE']
    #junção entre a tabela de região de saúde, trazendo descricao abreviada da regiao de saude 
    #e a quantidade de municipios na regiao de saude
    df_municipios = pd.merge(df_municipios, regiao_saude, how='left', on='CO_REGSAUD')

    #Merge do df df_municipios com o df populacao_regiao_saude, agregando o dado POPULACAO_REGIAO_SAUDE
    populacao_regiao_saude.columns = ['CO_REGSAUD', 'POPULACAO_REGIAO_SAUDE']
    df_municipios = pd.merge(df_municipios, populacao_regiao_saude, how='left', on='CO_REGSAUD')

    #Merge do df df_municipios com o df de classe de população, trazendo o valor da classe da população do município
    df_municipios = pd.merge(df_municipios, classe_populacao, how='left', left_on='CO_MUNICDV', right_on='CO_MUNICIP', copy=False)

    #Altera o nome da coluna código do município e dropa coluna desnecessária
    df_municipios.drop(columns='CO_MUNICIP_y', inplace=True)
    df_municipios.rename(columns={"CO_MUNICIP_x": "CO_MUNICIP"}, inplace=True)

    #filtra só os municípios da Paraíba, objeto do estudo
    df_municipios = df_municipios[df_municipios['CO_MUNICIP'].str.startswith(codigo_paraiba)]

    return df_municipios.loc[:,['CO_MUNICIP', 'IN_POBREZA', 'REGIAO_SAUDE', 'CLASSE_POP', 'QTD_MUNICIPIOS_REGIAO_SAUDE', 'POPULACAO_REGIAO_SAUDE']]

In [ ]:
def obtem_cnes():
    
    #Última atualização dos arquivos do CNES - 2019-12
    #Origem dos dados ftp://arpoador.datasus.gov.br/dissemin/publicos/CNES/200508_/Auxiliar/
    #http://cnes.datasus.gov.br/pages/downloads/arquivosBaseDados.jsp
    #carrega no df cnes_principal as informaçções do CNES contidas no arquivo tbEstabelecimento201912
    cnes_principal = carrega_arquivos_dataframe(local=os.path.join(base_data_path, estabelecimentos_folder_name), 
                                                nome='tbEstabelecimento201912', tipo = 'csv', encoding = 'utf-8') 
    #Origem dos dados ftp://arpoador.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/ST/
    #carrega no df cnes_estabelecimentos as informações contidas no arquivo STPB1912, que são os estabelecimentos que
    #compõem a rede de saúde do Estado da Paraíba em 2019 mês 12
    cnes_estabelecimentos = carrega_arquivos_dataframe(nome='STPB1912', local=os.path.join(base_data_path, estabelecimentos_folder_name)) 
    #Origem dos dados ftp://arpoador.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/LT/
    # carrega no df cnes_leitos os dados dos leitos habilitados no CNES da rede de saúde da Paraíba em 2019, mês 12
    cnes_leitos = carrega_arquivos_dataframe(nome='LTPB1912', local=os.path.join(base_data_path, leitos_folder_name))
    #Dicionário dos dados disponível em ftp://arpoador.datasus.gov.br/dissemin/publicos/CNES/200508_/doc/IT_CNES_1706.pdf

    #No arquivo tbEstabelecimento201912, há campos essenciais para a análise, por isso deve ser
    # considerada. Todavia, a tabela 'STPB1912.dbf' possuiu os estabelecimentos apenas da Paraíba, 
    # sendo assim, ao fazer o merge entre os dois dataframes com how='inner', realiza-se o filtro para o estado.
    cnes = pd.merge(cnes_principal, cnes_estabelecimentos, left_on='CO_CNES', right_on = 'CNES', how='inner')

    # O que importa é apenas determinar se o estabelecimento realiza atividades de ensino/pesquisa
    # O Domínio é "01";"UNIDADE UNIVERSITARIA" "02";"UNIDADE ESCOLA SUPERIOR ISOLADA" "03";"UNIDADE AUXILIAR DE ENSINO"
    # "04";"UNIDADE SEM ATIVIDADE DE ENSINO" "05";"HOSPITAL DE ENSINO"
    # fonte: tbAtividadeEnsino201912
    cnes['ATIVIDADE_ENSINO'] = cnes['ATIVIDAD'].map(lambda value: '0' if value == '04' else '1')
    # Dicionário para mapeamento do código do Tipo do Leito. A descrição
    #   foi extraída do site do CNES: http://cnes.datasus.gov.br/pages/estabelecimentos/consulta.jsp
    # tbAtributo, para todos os valores onde "CO_INDICADOR"="006"
    tp_leito = {'1':'LEITOS_CIRURGICOS',  '2':'LEITOS_CLINICOS',    '3':'LEITOS_COMPLEMENTARES', 
                '4':'LEITOS_OBSTETRICOS', '5':'LEITOS_PEDIATRICOS', '6':'LEITOS_OUTROS', 
                '7':'LEITOS_HOSPITAL_DIA'}
    cnes_leitos['TP_LEITO_DESCRICAO'] = cnes_leitos['TP_LEITO'].map(tp_leito)

    # Totaliza a quantidade de leitos existentes na entidade para se determinar
    #   o porte do estabelecimento
    # O comando realiza o somatório dos valores contidos na coluna QT_EXIST, para cada CNES, e as descrições dos leitos 
    # contidos em cada linha da coluna TP_LEITO_DESCRICAO viram colunas.
    cnes_leitos_total = pd.pivot_table(cnes_leitos, values='QT_EXIST', index='CNES', 
                                       columns='TP_LEITO_DESCRICAO', aggfunc=np.sum, dropna=False).fillna(0)

    # Estabelecimento do seguinte critério:
    #    Pequeno Porte - até 50 leitos
    #    Médio Porte = de 51 a 150,
    #    Grande Porte = de 151 até 500 leitos
    #    Porte Especial = Acima de 500 leitos
    # fonte: https://www.conass.org.br/consensus/armando-de-negri-o-papel-hospital-na-rede-de-atencao-saude/#:~:text=1)%20Porte%20do%20hospital%3A,de%20151%20a%20500%20leitos.
    def porte_hospitalar(qtd_leitos):
        if qtd_leitos < 51:
            porte = 'PEQUENO'
        elif qtd_leitos < 151:
            porte = 'MEDIO'
        elif qtd_leitos < 500:
            porte = 'GRANDE'
        else:
            porte = 'ESPECIAL'

        return porte
# Cria a coluna PORTE_HOSPITALAR no df cnes_leitos_total, recebendo o rótulo do PORTE HOSPITALAR, atribuído em função
# do número total de leitos (calculados pela função sum(axis=1)) e pela aplicacao da funcao porte_hospitalar.
    cnes_leitos_total['PORTE_HOSPITALAR'] = cnes_leitos_total.sum(axis=1).map(porte_hospitalar)

    #Serão utilizados apenas os seguintes atributos
    # Realiza a seleção das colunas 'CNES', 'CO_MUNICIPIO_GESTOR', 'TP_UNIDADE', 'CO_NATUREZA_JUR', 
    # 'NIV_DEP', 'TP_GESTAO', 'VINC_SUS', 'ATIVIDADE_ENSINO'
    cnes_selecao = cnes[['CNES', 'CO_MUNICIPIO_GESTOR', 'TP_UNIDADE', 
                         'CO_NATUREZA_JUR', 'NIV_DEP', 'TP_GESTAO', 'VINC_SUS', 
                         'ATIVIDADE_ENSINO']].copy()

    # Junção (semelhante ao LEFT INNER JOIN do SQL) dos df cnes_selecao e cnes_leitos_total.
    cnes_selecao = pd.merge(cnes_selecao, cnes_leitos_total, how = 'left', left_on = 'CNES', right_on = 'CNES' )

    # Após a junção, aquelas unidades de saúde que constam no df cnes_selecao e não constam no df cnes_leitos_total
    # terão todas as colunas que se referem a qtd de leitos com valores NaN. Portanto, para essas linhas,
    # preenche-as com valor zero.
    cnes_selecao[cnes_selecao.columns[8:-1]] = cnes_selecao[cnes_selecao.columns[8:-1]].fillna(0)

    # Supõe-se que as entidades que não tiverem leitos informados serão categorizadas como 
    # pequeno porte
    cnes_selecao['PORTE_HOSPITALAR'] = cnes_selecao['PORTE_HOSPITALAR'].fillna('PEQUENO')

    return cnes_selecao


In [ ]:
def obtem_cid10():
    cid = carrega_arquivos_dataframe(local=os.path.join(base_data_path, CID_folder_name), 
                                         nome = 'CID-10-CAPITULOS', tipo = 'csv', encoding='UTF-8')

        #carrega arquivo com os capítulos da CID10
        #cid = carrega_arquivos_dataframe(local=os.path.join(base_data_path, CID_folder_name), 
        #                                 nome = 'CID-10-CAPITULOS', tipo = 'csv', encoding='UTF-8')

        #Cria a sequencia de valores para realizar o merge com a tabela do SIHSUS
    categoria = {}
    #pega cada linha e seu índice respectivo e itera no for
    for index,row in cid.iterrows():
        #atribui a cat_inic o primeiro caracter (a letra da CID) armazenado na coluna CATINIC
        cat_inic = row['CATINIC'][0]
        #atribui a num_cat_inic o segundo caracter em diante (a parte numérica da CID) armazenado na coluna CATINIC
        num_cat_inic = int(row['CATINIC'][1:])
        #atribui a cat_fim o primeiro caracter (a letra da CID) armazenado na coluna CATFIM
        cat_fim = row['CATFIM'][0]
        #atribui a num_cat_fim o segundo caracter em diante (a parte numérica da CID) armazenado na coluna CATFIM
        num_cat_fim = int(row['CATFIM'][1:])
        #se a primeira letra da categoria inicial for igual à primeira letra da categoria fim 
        if cat_inic == cat_fim:
            categoria.update({''.join([cat_inic, str(cat).rjust(2, '0')]): row['DESCRABREV'] 
                                   for cat in range(num_cat_inic, num_cat_fim + 1)})
        #se a primeira letra da categoria inicial não for igual à primeira letra da categoria fim
        else:
            categoria.update({''.join([cat_inic, str(cat).rjust(2, '0')]): row['DESCRABREV'] 
                                   for cat in range(num_cat_inic, 100)})
            categoria.update({''.join([cat_fim, str(cat).rjust(2, '0')]): row['DESCRABREV'] 
                                   for cat in range(0, num_cat_fim+1)})

    mapeamento_cid = pd.DataFrame(list(categoria.items()), columns=['CATEGORIA_CID10', 'DESCRICAO_CID10'])
    
    return mapeamento_cid

In [ ]:
def relaciona_datasets(sih, cnes, municipio, cid):
    #Somente irá considerar as entidades ativas na data da extração
    df = pd.merge(sih, cnes, on = 'CNES')

    df = pd.merge(df, municipio, left_on = 'CO_MUNICIPIO_GESTOR', right_on = 'CO_MUNICIP')
    
    df['CAT_CID_DIAG_PRINC'] = df.DIAG_PRINC.str[0:3]
    df = pd.merge(df, cid, left_on = 'CAT_CID_DIAG_PRINC', right_on = 'CATEGORIA_CID10')
    
    return df

In [ ]:
#Transforma a idade do paciente para um valor em anos, realizando transformações de acordo com o campo COD_IDADE
def padroniza_idade(dataset):
    
    #O campo COD_IDADE armazena a unidade de medida da idade e tem o seguinte domínio:
    # 0: ignorada; 2: dias; 3: meses; 4: anos; 5: mais de 100 anos
    # Para valores de ignorada, dias e meses, colocou-se idade 0.
    # Para valores em anos, utilizou-se a própria idade apresentada
    # e para valores 5 (mais de 100 anos), atribui-se idade=100.
    def altera_idade(row):
        if row['COD_IDADE'] in ['0', '2', '3']:
            idade = 0
        elif row['COD_IDADE'] == '4':    
            idade = row['IDADE']
        else:    
            idade = 100
            
        return idade
        
    campo_idade_padronizado = pd.to_numeric(dataset[['COD_IDADE', 'IDADE']].apply(altera_idade, axis=1),
                                    downcast = 'integer')
    return campo_idade_padronizado

In [ ]:
#Cria o campo indicativo de UTI
def cria_indicativo_uti(dataset):
    # Se o campo UTI_MES_TO que armazena o total de diárias de UTI e o campo VAL_UTI que armazena o valor de UTI 
    # forem ambos maiores que zero, retorna 1, caso contrário retorna 0.
    campo_UTI = ((dataset['UTI_MES_TO'] > 0) & (dataset['VAL_UTI'] > 0))\
                    .map(lambda valor: 1 if valor==True else 0)
    return campo_UTI

In [ ]:
# função que calcula a razão entre o valor total da AIH e valor mediano, por procedimento e por indicativo de UTI
def calcula_razao(dataset):
    # Calcula o valor mediano do VAL_TOT das AIH, por PROC_REA e por UTI.
    valor_mediano_por_procedimento = dataset.groupby(['PROC_REA'], as_index = False)['VAL_TOT'].median()
    valor_mediano_por_procedimento.columns = ['PROC_REA', 'MEDIANA']
    #Junção entre o dataframe dataset e dataframe 
    dataset = dataset.merge(right=valor_mediano_por_procedimento, on=['PROC_REA'], how='inner')
    dataset['RAZAO'] = dataset['VAL_TOT']/dataset['MEDIANA']
    return dataset

In [ ]:
def calcula_zscore(dataset):
    def zscore(x):
        #calcula o desvio padrão
        sd = x.std()
        #evitando divisão por zero, se sd==0, acrescento 
        if math.isclose(sd, 0, rel_tol=1e-6): sd = 0.0000001
        #retorna (x-média)/(desvio-padrão)
        return (x-x.mean())/sd
    zscore_por_procedimento = dataset.groupby(['PROC_REA'], as_index = True)['VAL_TOT'].apply(zscore)
    #imputando 0 nos std que deram nan (N=1 e ddof=0 por default no pandas)
    zscore_por_procedimento.fillna(value=0, inplace=True)
    zscore_por_procedimento.rename("ZSCORE", inplace=True)
    #Junção entre o dataframe dataset e dataframe 
    dataset = dataset.merge(right=zscore_por_procedimento, left_index=True, right_index=True, how='inner')
    return dataset

In [ ]:
def calcula_outlier_IQR(dataset):
    def IQR(x):
        #calcula quantil 25
        Q1 = x.quantile(0.25)
        #calcula quantil 75
        Q3 = x.quantile(0.75)
        IQR = Q3 - Q1
        lista_outlier = []
        for index, value in x.items(): 
            if(value>Q3+(IQR*1.5)): 
                lista_outlier.append(1)
            else:
                lista_outlier.append(0)
        return pd.Series(lista_outlier, index = x.index) 
    # resultado do IQR outlier 
    IQR_por_procedimento = dataset.groupby(['PROC_REA'], as_index = True)['VAL_TOT'].apply(IQR)
    IQR_por_procedimento.rename("OUTLIER_IQR", inplace=True)
    #Junção entre o dataframe dataset e dataframe 
    dataset = dataset.merge(right=IQR_por_procedimento, left_index=True, right_index=True, how='inner')
    return dataset

In [ ]:
# Função que retorna uma série contendo o quinto caracter da coluna N_AIH
# Este caracter corresponde ao número de referência da AIH, em que
# 1 é utilizado para AIH geral, 2 para específica da CNRAC e 5 para
# procedimentos cirúrgicos eletivos de média complexidade
# fonte: Uma_analise_da_base_de_dados_do_sistema_de_informacao_hospitalar.pdf
def origem_autorizacao(dataset):
    coluna_origem_autorizacao = dataset['N_AIH'].str[4]
    return coluna_origem_autorizacao

In [ ]:
# Função que compara o código do município de residência do paciente com o
# código do município do gestor do estabelecimento.
# Caso eles sejam iguais, retorna 1, caso contrário, retorna 0.
# Retorna uma série com esses 0 ou 1.
def cria_campo_residencia_local(dataset):
    coluna_reside_mun =  (dataset['MUNIC_RES'] == dataset['CO_MUNICIPIO_GESTOR'])\
                                            .map(lambda v: 1 if v==True else 0)
    
    return coluna_reside_mun

In [ ]:
# Função que retorna uma série contendo a transformação do campo ESPEC do SIH.
# O campo ESPEC armazena a especialidade do leito utilizado na internação.
# Domínio = 01-Cirurgia, 02-Obstetrícia, 03-Clínica médica, 04-Pacientes sob cuidados prolongados
# 05-Psiquiatria, 06-Tisiologia, 07-Pediatria, 08-Reabilitação, 09-Psiquiatria em hospital-dia
# Para os valores maiores ou igual a 10, não encontrei explicação.
# Basicamente, para as especialidados cuja frequência relativa é maior
def transforma_especialidade(dataset):    
    campo_especialidade = pd.to_numeric(dataset['ESPEC'], downcast = 'integer')
    
    especialidades = pd.DataFrame(campo_especialidade.value_counts(normalize=True))
    
    especialidades = especialidades[(especialidades['ESPEC'] > 0.01) & (especialidades.index <= 9)]
    
    campo_especialidade = campo_especialidade.map(lambda v: v if v in especialidades.index else 10)
    
    return campo_especialidade

In [ ]:
def get_natureza_juridica_simplificada(dataset):
    return dataset['CO_NATUREZA_JUR'].str.slice(start=0, stop=1, step=None)

### Execução das funções que realizarão a leitura dos dados.

In [ ]:
# A partir de agora, começa-se a chamar as funções criadas.
#Carrega a produção hospitalar bruta
dados_producao_hospitalar_bruta = obtem_producao_hospitalar()

In [ ]:
# Carrega dados dos municípios
dados_municipios = ler_arquivos_territoriais()

In [ ]:
# Carrega dados do CNES
dados_estabelecimentos_CNES = obtem_cnes()

In [ ]:
# Carrega dados da CID
dados_CID = obtem_cid10()

In [ ]:
#Quantidade de linhas e colunas em cada dataset.
print(f'SIH: Registros {dados_producao_hospitalar_bruta.shape[0]} - colunas {dados_producao_hospitalar_bruta.shape[1]}')
print(f'Municípios: Registros {dados_municipios.shape[0]} - colunas {dados_municipios.shape[1]}')
print(f'Estabelecimentos de Saúde: Registros {dados_estabelecimentos_CNES.shape[0]} - colunas {dados_estabelecimentos_CNES.shape[1]}')
print(f'CID-10: Registros {dados_CID.shape[0]} - colunas {dados_CID.shape[1]}')

In [ ]:
#Retorna a quantidade de linhas duplicadas no SIH
print(f'Quantidade de linhas duplicadas no SIH = {dados_producao_hospitalar_bruta.duplicated().sum()}')

### Realiza a junção e faz pequenas alterações em algumas colunas do dataset.

In [ ]:
# Salva em dataset_global os datasets relativos à produção hospitalar, dos estabelecimentos de saúde, dos municípios e da CID
dataset_global = relaciona_datasets(dados_producao_hospitalar_bruta, dados_estabelecimentos_CNES, dados_municipios, dados_CID)

In [ ]:
#Cria uma coluna denominada UTI na dataset_global que indica 
#se houve utilização de UTI (valor=1, caso contrário valor=0) na internação hospitalar.
dataset_global['UTI'] = cria_indicativo_uti(dataset_global)

In [ ]:
# Preenche com o valor 0.0 os valores que estão vazios nas colunas
# VAL_TOT e VAL_UTI
dataset_global['VAL_TOT'].fillna(0.0, inplace = True)
dataset_global['VAL_UTI'].fillna(0.0, inplace = True)

In [ ]:
#Cria o campo "TOTAL_GERAL", já que os valores de UTI e o restante são separados
# dataset_global['TOTAL_GERAL'] = dataset_global['VAL_TOT'] + dataset_global['VAL_UTI']

In [ ]:
#Exclui os registros em que VAL_TOT=0, pois AIH sem custo não interessam ao objetivo do estudo.
dataset_global.drop(labels=dataset_global.query('VAL_TOT == 0').index.values, axis=0, inplace=True)

In [ ]:
#dataset_global = calcula_razao(dataset_global)

In [ ]:
#dataset_global = calcula_zscore(dataset_global)

In [ ]:
#dataset_global = calcula_outlier_IQR(dataset_global)

In [ ]:
dataset_global['ORIGEM_AUTORIZACAO'] = origem_autorizacao(dataset_global)

In [ ]:
dataset_global['RESIDE_LOCAL_ATENDIMENTO'] = cria_campo_residencia_local(dataset_global)

In [ ]:
dataset_global['IDADE_PADRONIZADA'] = padroniza_idade(dataset_global)

In [ ]:
dataset_global['ESPEC'] = transforma_especialidade(dataset_global)

In [ ]:
dataset_global['CO_NATUREZA_JUR'] = get_natureza_juridica_simplificada(dataset_global)

Vamos renomear as colunas para melhorar a inteligibilidade.

In [ ]:
colunas = {'ESPEC': 'especialidade_leito',
           'IDENT': 'tipo_AIH',
           'SEXO': 'sexo',
           'MARCA_UTI': 'tipo_UTI',
           'GESTAO': 'gestao_hospitalar',
           'IDADE_PADRONIZADA': 'idade',
           'MORTE': 'obito',
           'CAR_INT': 'carater_internacao',
           'COMPLEX': 'complexidade',
           'FINANC': 'tipo_financiamento',
           'TP_UNIDADE': 'tipo_unidade',
           'CO_NATUREZA_JUR': 'natureza_juridica_entidade_simplificada',
           'NIV_DEP': 'grau_dependencia_entidade',
           'ATIVIDADE_ENSINO': 'atividade_ensino',
           'PORTE_HOSPITALAR': 'porte_hospitalar',
           'IN_POBREZA': 'municipio_extrema_probreza',
           'QTD_MUNICIPIOS_REGIAO_SAUDE': 'qtd_municipios_regiao_saude',
           'CLASSE_POP': 'classe_populacao',
           'DESCRICAO_CID10': 'capitulo_CID',
           'UTI': 'internacao_com_UTI',
           'PROC_REA':'procedimento_realizado',
           'ORIGEM_AUTORIZACAO': 'origem_autorizacao',
           'RESIDE_LOCAL_ATENDIMENTO': 'reside_local_atendimento',
           'VAL_TOT': 'valor_total'}

dataset_global = dataset_global.rename(columns=colunas, errors='raise')

Vamos ver como estão os tipos de dados.

In [ ]:
dataset_global.info()

Vamos alterar o tipo dos dados para os tipos corretos.

In [ ]:
dataset_global = dataset_global.astype({"especialidade_leito":'category'
                                        ,"tipo_AIH":'category'
                                        ,'sexo':'category'
                                        ,'gestao_hospitalar':'category'
                                        ,'idade':'int'
                                        ,'obito':'category'
                                        ,'carater_internacao':'category'
                                        ,'complexidade':'category'
                                        ,'tipo_financiamento':'category'
                                        ,'tipo_unidade':'category'
                                        ,'natureza_juridica_entidade_simplificada':'category'
                                        ,'grau_dependencia_entidade':'category'
                                        ,'atividade_ensino':'category'
                                        ,'porte_hospitalar':'category'
                                        ,'municipio_extrema_probreza':'category'
                                        ,'qtd_municipios_regiao_saude':'int'
                                        ,'classe_populacao':'category'
                                        ,'capitulo_CID':'category'
                                        ,'internacao_com_UTI':'category'
                                        ,'procedimento_realizado':'category'
                                        ,'origem_autorizacao':'category'
                                        ,'reside_local_atendimento':'category'
                                        ,'valor_total':'float'}) 

Salvando os índices do meu DF com os respectivos números de AIH para posterior necessidade de rastreamento das AIH.

In [ ]:
df_index_naih = dataset_global[['N_AIH']].copy()

#Salva o Dataframe no disco. Caso já esteja salvo, apaga-o antes.
if os.path.exists(os.path.join(base_data_path,joblibs_folder_name,joblib_n_aih)):
    os.remove(os.path.join(base_data_path,joblibs_folder_name,joblib_n_aih))
dump(df_index_naih, os.path.join(base_data_path,joblibs_folder_name,joblib_n_aih))

### DATASET FINAL

- __Campo 'especialidade_leito'__: Especialidade do Leito utilizado na AIH: 01-Cirurgia; 02-Obstetrícia; 03-Clínica médica; 04-Pacientes sob cuidados prolongados; 05-Psiquiatria; 06-Tisiologia; 07-Pediatria; 08-Reabilitação; 09-Psiquiatria em hospital-dia. Não foi encontrado justicativa para o código 10. Todavia, os campos em que tiveram frequência relativa abaixo de 1% ou que os casos em que não se encontrou documentação/explicação para aqueles valores (10 ou acima) foram colocados na categoria "10 - OUTROS". Origem: Campo ESPEC do RD SIH.
- __Campo 'tipo_AIH'__: Identificação do tipo da AIH - Existem dois tipos de AIH: Tipo 1: inicial e Tipo 5: continuidade. Origem: campo IDENT do RD SIH.
- __Campo 'sexo'__: Sexo do Paciente - 1 - masculino, 3 - feminino. Origem: Campo SEXO do RD SIH.
- __Campo 'gestao_hospitalar'__: Indica o tipo de gestão do hospital: 0 - gestão estadual; 1 - gestão plena municipal; 2 - gestão plena estadual; demais: não definida. Origem: Campo Gestao do RD SIH.
- __Campo 'idade'__ : Idade transformada do paciente, uma vez que o campo 'COD_IDADE' tem a unidade de medida da idade, as idades que estão em dias ou meses foram transformadas em 0 anos, e aquelas acima de 100 anos, foram fixadas em 100 anos. Origem: combinação dos campos COD_IDADE e IDADE do RD SIH.
- __Campo 'obito'__: 1 - indica o óbito do paciente. Origem: Campo MORTE do RD SIH. 
- __Campo 'carater_internacao'__: Caráter de internação: 01-Eletivo; 02-Urgência; 03-Acidente no local de trabalho ou a serviço da empresa; 04-Acidente no trajeto para o trabalho; 05-Outros tipos de acidente de transito; 06-Outros tipos de lesões e envenenamentos por agentes químicos ou físicos. Origem: Campo CAR_INT do RD SIH.
- __Campo 'complexidade'__: 0 - Não se aplica, 1 - Atenção Básica Complexidade, 2 - Média Complexidade, 3 - Alta Complexidade. Origem: Campo COMPLEX do RD SIH.
- __Campo 'tipo_financiamento'__: Tipo de Financiamento - Conforme SIGTAP, Tabelas, Relacionadas ao procedimento, Complementares, "Tipo de Financiamento" do SIGTAP - 01 - Atenção Básica (PAB); 02 - Assistência Farmacêutica; 04- Fundo de Ações Estratégicas e Compensações (FAEC); 05-Incentivo - MAC; 06- Média e Alta Complexidade (MAC); 07 - Vigilância em Saúde; 08 - Gestão do SUS. Origem: campo FINANC do RD SIH.
- __Campo 'tipo_unidade'__: Código do tipo de unidade de saúde. Exemplo: "05"="HOSPITAL GERAL" e "07"="HOSPITAL ESPECIALIZADO". Tabela completa do domínio desta coluna encontra-se na Tabela tbTipoUnidade do CNES. 
- __Campo 'natureza_juridica_entidade_simplificada'__: Primeiro caracter do código da natureza Jurídica do Estabelecimento de Saúde conforme comissão nacional de classificação. Se 1=Administração Pública, se 2=Entidades Privadas, se 3=Entidades sem fins lucrativos. O domínio completo pode ser visto na Tabela tbNaturezaJuridica do CNES. Origem: campo CO_NATUREZA_JUR da tabela tbEstabelecimento201912 do CNES.
- __Campo 'grau_dependencia_entidade'__: Grau de dependência, onde: 1-Individual 3-Mantida. Campo NIV_DEP da tabela tbEstabelecimento201912 do CNES.
- __Campo 'atividade_ensino'__: 1 - Entidade de saúde realiza atividade de ensino. 0 - Entidade de saúde não realiza atividade de ensino. Campo ATIVIDADE_ENSINO da tabela tbEstabelecimento201912 do CNES.
- __Campo 'porte_hospitalar'__: Pequeno Porte - até 50 leitos; Médio Porte - de 51 a 150; Grande Porte - de 151 até 500 leitos; Porte Especial - Acima de 500 leitos. Origem: linhas do arquivo LTPB1912 do CNES. Ver função porte_hospitalar(qtd_leitos) para maiores detalhes.
- __Campo 'municipio_extrema_probreza'__: 1 - Municípios em Extrema Pobreza. 0 - aqueles que não estão em extrema pobreza. Origem: Campo IN_POBREZA da tb_municip do IBGE.
- __Campo 'qtd_municipios_regiao_saude'__: Tabela TB_REGSAUDE (IBGE) - Região de saúde é o espaço geográfico contínuo constituído por agrupamentos de municípios limítrofes, delimitado a partir de identidades culturais, econômicas e sociais e de redes de comunicação e infraestrutura de transportes compartilhados, com a finalidade de integrar a organização, o planejamento e a execução de ações e serviços de saúde. Para o campo, será considerada a quantidade de municípios na região de saúde. 
- __Campo 'classe_populacao'__: Classe da população presente no município onde está situado o estabelecimento de saúde. Classes = 1 - Até 5000; 2 - 5001 até 10000; 3 - 10001 até 20000; 4 - 20001 até 50000; 5 - 50001 até 100000; 6 - 100001 até 500000; 7 - Maior que 500000. Origem: coluna CLASSE POP do arquivo ibge_populacao.csv. 
- __Campo 'capitulo_CID'__: Nome do capítulo referente à CID do diagnóstico principal : I. Algumas doenças infecciosas e parasitárias; II. Neoplasias (tumores) III. Doenças sangue órgãos hemat e transt imunitár; IV. Doenças endócrinas nutricionais e metabólicas; V. Transtornos mentais e comportamentais; VI. Doenças do sistema nervoso; VII. Doenças do olho e anexos; VIII.Doenças do ouvido e da apófise mastóide; IX. Doenças do aparelho circulatório; X. Doenças do aparelho respiratório; XI. Doenças do aparelho digestivo; XII. Doenças da pele e do tecido subcutâneo; XIII.Doenças sist osteomuscular e tec conjuntivo; XIV. Doenças do aparelho geniturinário; XV. Gravidez parto e puerpério; XVI. Algumas afec originadas no período perinatal; XVII.Malf cong deformid e anomalias cromossômicas; XVIII.Sint sinais e achad anorm ex clín e laborat; XIX. Lesões enven e alg out conseq causas externas; XX. Causas externas de morbidade e mortalidade; XXI. Contatos com serviços de saúde; XXII.Códigos para propósitos especiais. Origem: CID.
- __Campo 'internacao_com_UTI'__: 1 - indica se o procedimento utilizou UTI. Origem: campos UTI_MES_TO (Total de dias de UTI durante a internação) e VAL_UTI (Valor referente aos gastos em UTI) do RD SIH.
- __Campo 'procedimento_realizado'__: Procedimento principal realizado, conforme SIGTAP. Origem: coluna PROC_REA do RD SIH.
- __Campo 'origem_autorizacao'__: 1 - para identificar que a autorização é de Internação (AIH) - uso geral; 3 - para identificar que a numeração é de internação (AIH) específica da CNRAC; 5 - para identificar que a autorização é de internação (AIH) específica para a estratégia de aumento do acesso aos Procedimentos Cirúrgicos Eletivos no âmbito do Sistema Único de Saúde (SUS), seja componente I, II ou III, definidos pela Portaria GM/MS nº 1.340, de 29 de junho de 2012 e legislação correlata. Origem: 5 dígito do campo N_AIH do RD SIH.
- __Campo 'reside_local_atendimento'__: 1 - Paciente reside no município onde está a entidade de saúde. Origem: comparação entre o campo MUNIC_RES do RD SIH e CO_MUNICIPIO_GESTOR da tabela tbEstabelecimento201912 do CNES.
- __Campo 'valor_total'__: É o valor total da internação. Origem: campo VAL_TOT do RD SIH.
- __Campo 'razao_valor_mediana'__: Razão entre o valor total da AIH e a mediana da tupla procedimento, uti. 
- __Campo 'zscore'__: ZScore do valor total da AIH, considerando a tupla procedimento, uti.
- __Campo 'outlier_IQR'__: é outlier pelo critério IQR=1.5 na coluna valor total da AIH, considerando cada grupo como sendo a tupla procedimento, uti.

In [ ]:
### Dicionário do Modelo Final
dados_preparados = dataset_global[['especialidade_leito'
                                   ,'tipo_AIH'
                                   ,'sexo'
                                   ,'gestao_hospitalar'
                                   ,'idade'
                                   ,'obito'
                                   ,'carater_internacao'
                                   ,'complexidade'
                                   ,'tipo_financiamento'
                                   ,'tipo_unidade'
                                   ,'natureza_juridica_entidade_simplificada'
                                   ,'grau_dependencia_entidade'
                                   ,'atividade_ensino'
                                   ,'porte_hospitalar'
                                   ,'municipio_extrema_probreza'
                                   ,'qtd_municipios_regiao_saude'
                                   ,'classe_populacao'
                                   ,'capitulo_CID'
                                   ,'internacao_com_UTI'
                                   ,'procedimento_realizado'
                                   ,'origem_autorizacao'
                                   ,'reside_local_atendimento'
                                   ,'valor_total']].copy()

In [ ]:
#Salva o Dataframe no disco. Caso já esteja salvo, apaga-o antes.
if os.path.exists(os.path.join(base_data_path,joblibs_folder_name,joblib_file_name)):
    os.remove(os.path.join(base_data_path,joblibs_folder_name,joblib_file_name))
dump(dados_preparados, os.path.join(base_data_path,joblibs_folder_name,joblib_file_name))

In [ ]:
# %run dbf_to_parquet.py
dados_preparados.shape